In [1]:
import os
import json

In [2]:
filepath = '../data/harem/CDPrimeiroHAREMprimeiroevento.xml'

In [3]:
from lxml import etree

In [4]:
SELECTIVE_CATEGS = [
    'PESSOA',
    'ORGANIZACAO',
    'LOCAL',
    'TEMPO',
    'VALOR',
]

In [5]:
OTHER_CATEGS =  [
    'ABSTRACCAO',
    'ACONTECIMENTO',
    'COISA',
    'OBRA',
    'OUTRO',
    'OBJECTO'
]

In [6]:
MAP = {
    'O': 'O',
    'PESSOA': 'PER',
    'ORGANIZACAO': 'ORG',
    'LOCAL': 'LOC',
    'TEMPO': 'TEMP',
    'VALOR': 'VAL',
}
MAP.update({
    k:'O' for k in OTHER_CATEGS
})

In [7]:
import xml.etree.ElementTree as ET
tree = ET.parse(filepath)
root = tree.getroot()

In [8]:
root

<Element 'colHAREM' at 0x7f537cc0e1d0>

In [9]:
def child_to_samples(child):
    entities = []
    tags = []
    for c in child:
        if not len(c.items()):
            continue
        categ = c.items()[1][1]
        text = c.text

        entities.append((text, categ))
        tags.append(categ)

    entity_id = 0
    rows = []
    for text in child.itertext():
        if entity_id < len(entities):
            ent, tag = entities[entity_id]
            if text == ent:
                label = tag
                entity_id += 1
            else:
                label = 'O'
        else:
            label = 'O'

        rows.append((text, label))
        
    sentences = []
    last = 0
    for i, entry in enumerate(rows):
        if '\n' in entry[0]:
            sentences.append(rows[last:i+1])
            last = i + 1

    return sentences

def sample_to_text(sample):
    final_sample = []
    for text, tag in sample:
        text = text.strip()
        if not len(text):
            continue
        label = MAP[tag.split('|')[0]]

        words = text.split(' ')
        if label != 'O':
            labels = [f'B-{label}'] + [f'I-{label}'] * (len(words) - 1)
        else:
            labels = ['O'] * len(words)

        for w, l in zip(words, labels):
            final_sample.append(' '.join([w, l]))
    
    return '\n'.join(final_sample)

In [10]:
final_samples = []
for child in root:
    ss = child_to_samples(child)
    samples = []
    for s in ss:
        text = sample_to_text(s)
        if len(text):
            samples.append(text) 
    
    final_samples.append('\n'.join(samples))
    

In [11]:
final_text = '\n\n'.join(final_samples)

In [12]:
with open('../data/harem/primeiroHarem.txt', 'w') as f:
    f.write(final_text)